***Importing Essential Libraries***

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
import os
from skimage.feature.texture import graycomatrix, graycoprops, local_binary_pattern
import pywt

***Feature Extraction***

In [ ]:
phi = 1.618  # golden ratio

# --- Divide a line segment using the golden ratio ---
def divide_point(p1, p2, ratio=phi):
    x = int((p1[0] + ratio * p2[0]) / (1 + ratio))
    y = int((p1[1] + ratio * p2[1]) / (1 + ratio))
    return (x, y)

# --- Preprocessing: create a square-padded version of the image ---
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    if img is None:
        return None
    h, w, _ = img.shape
    side = max(h, w)
    square_img = np.full((side, side, 3), (0, 0, 0), dtype=np.uint8)
    x_offset = (side - w) // 2
    y_offset = (side - h) // 2
    square_img[y_offset:y_offset+h, x_offset:x_offset+w] = img
    return square_img

# --- Compute statistics from a triangular region ---
def extract_triangle_features(img, pts):
    mask = np.zeros(img.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [np.array(pts)], 255)
    block = cv2.bitwise_and(img, img, mask=mask)
    gray = cv2.cvtColor(block, cv2.COLOR_BGR2GRAY)
    pixels = gray[gray > 0]
    if len(pixels) == 0:
        return [0, 0, 0, 0, 0]
    return [np.mean(pixels), np.median(pixels), np.std(pixels), skew(pixels), kurtosis(pixels)]

# --- Perform golden ratio subdivision: return 4 sub-triangles----
def golden_subdivide(A, B, C):
    D = divide_point(A, C, phi)
    E = divide_point(B, D, phi)
    F = divide_point(C, E, phi)
    return {
        "ABD": [A, B, D],
        "BCE": [B, C, E],
        "DFC": [D, F, C],
        "DEF": [D, E, F]
    }

# --- Construct the initial 6 triangles forming a hexagon around a circle ---
def split_into_triangles(img):
    h, w, _ = img.shape
    cx, cy = w // 2, h // 2
    radius = int(np.sqrt((w/2)**2 + (h/2)**2))
    hexagon = []
    for i in range(6):
        angle = np.deg2rad(60 * i - 30)
        x = int(cx + radius * np.cos(angle))
        y = int(cy + radius * np.sin(angle))
        hexagon.append((x, y))
    triangles = []
    for i in range(6):
        pt1, pt2, pt3 = (cx, cy), hexagon[i], hexagon[(i+1) % 6]
        triangles.append([pt1, pt2, pt3])
    return triangles, (cx, cy), radius, hexagon

# --- Box-counting fractal dimension ---
def fractal_dimension(Z, threshold=0.9):
    Z = (Z < threshold * Z.max())
    sizes = 2 ** np.arange(1, int(np.log2(min(Z.shape))) )
    counts = []
    for size in sizes:
        S = np.add.reduceat(
            np.add.reduceat(Z, np.arange(0, Z.shape[0], size), axis=0),
            np.arange(0, Z.shape[1], size), axis=1)
        counts.append(np.sum(S > 0))
    coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
    return -coeffs[0]


# --- Lacunarity ---
def lacunarity(Z, box_sizes=[2,4,8]):
    Z = (Z > 0).astype(np.uint8)
    lac = []
    for size in box_sizes:
        S = np.add.reduceat(
            np.add.reduceat(Z, np.arange(0, Z.shape[0], size), axis=0),
            np.arange(0, Z.shape[1], size), axis=1)
        mean = np.mean(S)
        var = np.var(S)
        if mean > 0:
            lac.append(var / (mean**2))
        else:
            lac.append(0)
    return lac


# --- Wavelet Energy ---
def wavelet_energy(Z, wavelet='db2', level=2):
    coeffs = pywt.wavedec2(Z, wavelet, level=level)
    energies = []
    for c in coeffs[1:]:
        for band in c:
            energies.append(np.sum(np.square(band)))
    return energies

def process_image(img_path, label):
    img = preprocess_image(img_path)
    if img is None:
        return None

    triangles, _, _, _ = split_into_triangles(img)
    feats_img = {}

    for i, tri in enumerate(triangles):
        subtris = golden_subdivide(tri[1], tri[2], tri[0])
        for j, (name, subtri) in enumerate(subtris.items(), start=1):

            # ---- Basic Stats ---- #
            feats = extract_triangle_features(img, subtri)
            feats_img[f"T{i+1}_{name}_mean"] = feats[0]
            feats_img[f"T{i+1}_{name}_median"] = feats[1]
            feats_img[f"T{i+1}_{name}_std"] = feats[2]
            feats_img[f"T{i+1}_{name}_skew"] = feats[3]
            feats_img[f"T{i+1}_{name}_kurt"] = feats[4]

            # ---- GLCM Features ---- #
            mask = np.zeros(img.shape[:2], dtype=np.uint8)
            pts = np.array([subtri], np.int32)
            cv2.fillPoly(mask, [pts], 255)

            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            block = cv2.bitwise_and(gray, gray, mask=mask)

            pixels = block[mask == 255]

            if pixels.size > 0:
                block_u8 = np.uint8(pixels.reshape(-1, 1))
                glcm = graycomatrix(block_u8,
                            distances=[1],
                            angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                            symmetric=True, normed=True)

                feats_img[f"T{i+1}_{name}_glcm_contrast"] = graycoprops(glcm, 'contrast').mean()
                feats_img[f"T{i+1}_{name}_glcm_correlation"] = graycoprops(glcm, 'correlation').mean()
                feats_img[f"T{i+1}_{name}_glcm_energy"] = graycoprops(glcm, 'energy').mean()
                feats_img[f"T{i+1}_{name}_glcm_homogeneity"] = graycoprops(glcm, 'homogeneity').mean()
            else:
                feats_img[f"T{i+1}_{name}_glcm_contrast"] = 0
                feats_img[f"T{i+1}_{name}_glcm_correlation"] = 0
                feats_img[f"T{i+1}_{name}_glcm_energy"] = 0
                feats_img[f"T{i+1}_{name}_glcm_homogeneity"] = 0

            lbp = local_binary_pattern(block, P=8, R=1, method="uniform")
            lbp_mean = lbp.mean()
            lbp_hist, _ = np.histogram(lbp.ravel(),
                                       bins=np.arange(0, 8+3),
                                       range=(0, 8+2),
                                       density=True)

            feats_img[f"T{i+1}_{name}_lbp_mean"] = lbp_mean
            for k, v in enumerate(lbp_hist):
                feats_img[f"T{i+1}_{name}_lbp_hist_{k}"] = v

            # ---- Edge Density ---- #
            edges = cv2.Canny(block, 100, 200)
            edge_density = np.sum(edges > 0) / float(block.size)
            feats_img[f"T{i+1}_{name}_edge_density"] = edge_density

            # ---- Gradient Stats ---- #
            grad_x = cv2.Sobel(block, cv2.CV_64F, 1, 0, ksize=3)
            grad_y = cv2.Sobel(block, cv2.CV_64F, 0, 1, ksize=3)
            grad_mag = np.sqrt(grad_x**2 + grad_y**2)

            feats_img[f"T{i+1}_{name}_gradient_mean"] = grad_mag.mean()
            feats_img[f"T{i+1}_{name}_gradient_std"] = grad_mag.std()

            # ---- Fractal Dimension ---- #
            try:
                fd = fractal_dimension(block)
            except:
                fd = 0
            feats_img[f"T{i+1}_{name}_fractal_dim"] = fd

            # ---- Lacunarity ---- #
            lac_vals = lacunarity(block, box_sizes=[2,4,8])
            for k, v in enumerate(lac_vals):
                feats_img[f"T{i+1}_{name}_lacunarity_r{[2,4,8][k]}"] = v

            # ---- Wavelet Energy ---- #
            wavelet_vals = wavelet_energy(block, wavelet='db2', level=2)
            for k, v in enumerate(wavelet_vals):
                feats_img[f"T{i+1}_{name}_wavelet_energy_{k}"] = v

    feats_img["label"] = label
    return feats_img


# --- Creating a CSV Dataset ---
def process_dataset(dataset_path, save_csv=True):
    label_map = {"notumor":0, "glioma":1, "meningioma":2, "pituitary":3}
    all_features = []
    for label in label_map.keys():
        folder = os.path.join(dataset_path, label)
        if not os.path.exists(folder):
            continue
        for file in os.listdir(folder):
            if file.lower().endswith((".jpg", ".jpeg", ".png", ".jpeg")):
                img_path = os.path.join(folder, file)
                feats = process_image(img_path, label_map[label])
                if feats is not None:
                    all_features.append(feats)
    df = pd.DataFrame(all_features)
    if save_csv:
        df.to_csv("dataset_features_golden.csv", index=False)
    return df

# --- Code to visualise the whirling triangle in image ---
def visualize_shapes(img_path):
    img = preprocess_image(img_path)
    h, w, _ = img.shape
    radius = int(np.sqrt((w/2)**2 + (h/2)**2))
    canvas_size = 2 * radius + 50
    canvas = np.full((canvas_size, canvas_size, 3), (50, 50, 50), dtype=np.uint8)
    x_offset = (canvas_size - w) // 2
    y_offset = (canvas_size - h) // 2
    canvas[y_offset:y_offset+h, x_offset:x_offset+w] = img
    cx, cy = canvas_size // 2, canvas_size // 2

    cv2.circle(canvas, (cx, cy), radius, (0, 255, 0), 3)
    hexagon = []
    for i in range(6):
        angle = np.deg2rad(60 * i - 30)
        x = int(cx + radius * np.cos(angle))
        y = int(cy + radius * np.sin(angle))
        hexagon.append((x, y))
    cv2.polylines(canvas, [np.array(hexagon, np.int32)], isClosed=True, color=(0, 0, 255), thickness=3)

    colors = [(255,255,0),(0,255,255),(255,0,255),(0,128,255)]
    for i in range(6):
        pt1, pt2, pt3 = (cx, cy), hexagon[i], hexagon[(i+1) % 6]
        cv2.polylines(canvas, [np.array([pt1, pt2, pt3], np.int32)], True, (255, 255, 255), 3)
        subtris = golden_subdivide(pt2, pt3, pt1)
        for j, subtri in enumerate(subtris.values()):
            cv2.polylines(canvas, [np.array(subtri, np.int32)], True, colors[j % len(colors)], 3)

    plt.figure(figsize=(8,8))
    plt.imshow(cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()

dataset_path = "E:/Research Papers/New folder/Codes/Brain Tumour Dataset"


In [ ]:
df = process_dataset(dataset_path, save_csv=True)
print("CSV saved: braintumor_features.csv")

***Visualising the different features in the image***

In [ ]:
def visualize_glcm_contrast(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    glcm = graycomatrix(img, distances=[1], angles=[0], symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')

    plt.figure(figsize=(5,5))
    plt.imshow(contrast, cmap='hot')
    plt.colorbar()
    plt.title("GLCM Contrast Map")
    plt.axis("off")
    plt.show()

def visualize_lbp(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    lbp = local_binary_pattern(img, P=8, R=1, method='uniform')

    plt.figure(figsize=(6,6))
    plt.imshow(lbp, cmap='gray')
    plt.title("Local Binary Pattern (LBP)")
    plt.axis("off")
    plt.show()

def visualize_edges(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    edges = cv2.Canny(img, 100, 200)

    plt.figure(figsize=(6,6))
    plt.imshow(edges, cmap='gray')
    plt.title("Edge Map (Canny)")
    plt.axis("off")
    plt.show()

def visualize_fractal_lacunarity(img_path, threshold=128):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    Z = (img > threshold).astype(int)

    plt.figure(figsize=(6,6))
    plt.imshow(Z, cmap='gray')
    plt.title("Binary Tumor Block (Fractal / Lacunarity)")
    plt.axis("off")
    plt.show()

def visualize_wavelet_energy(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    coeffs2 = pywt.wavedec2(img, 'db2', level=2)
    cA2, (cH2, cV2, cD2), (cH1, cV1, cD1) = coeffs2

    energy_map = np.square(cH2) + np.square(cV2) + np.square(cD2)

    plt.figure(figsize=(6,6))
    plt.imshow(energy_map, cmap='inferno')
    plt.title("Wavelet Energy Map (Level 2)")
    plt.axis("off")
    plt.show()

def visualize_gradient(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    gx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    gy = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
    grad_mag = np.sqrt(gx**2 + gy**2)
    plt.figure(figsize=(6,6))
    plt.imshow(grad_mag, cmap='inferno')
    plt.title("Sobel Gradient Magnitude")
    plt.axis("off")
    plt.show()

In [ ]:
img_path = "Brain Tumour Dataset/Te-me_0010.jpg"

visualize_shapes(img_path)
visualize_lbp(img_path)
visualize_edges(img_path)
visualize_gradient(img_path)
visualize_glcm_contrast(img_path)
visualize_fractal_lacunarity(img_path)
visualize_wavelet_energy(img_path

***Preprocessing The Dataset And Applying PCA***

In [ ]:
df = df.fillna(df.mean(numeric_only=True))

X = df.drop("label", axis=1).values
y = df["label"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

***Training The Machine Learning Models Using The Generated Dataset***

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from Pyfhel import Pyfhel, PyCtxt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM (RBF)": SVC(kernel='rbf', probability=True),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100),
    "AdaBoost": AdaBoostClassifier(n_estimators=100),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB(),
    "Decision Tree": DecisionTreeClassifier(),
    "XGBoost": XGBClassifier(eval_metric='mlogloss', use_label_encoder=False),
    "LightGBM": LGBMClassifier(),
    "CatBoost": CatBoostClassifier(verbose=0)
}

# --- Initialize Pyfhel for Homomorphic Encryption ---
HE = Pyfhel()
HE.contextGen(p=65537, m=4096, base=2)
HE.keyGen()

def encrypt_matrix(X):
    """Encrypt a 2D numpy array feature-wise."""
    X_enc = []
    for row in X:
        X_enc.append([HE.encryptFrac(val) for val in row])
    return X_enc

def decrypt_matrix(X_enc):
    """Decrypt a 2D encrypted matrix."""
    X_dec = []
    for row in X_enc:
        X_dec.append([HE.decryptFrac(val) for val in row])
    return np.array(X_dec)

results = []

for name, model in models.items():
    # --- Plaintext ---
    t0 = time.time()
    model.fit(X_train_pca, y_train)
    train_time = time.time() - t0

    t1 = time.time()
    y_pred = model.predict(X_test_pca)
    test_time = time.time() - t1

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    prec = precision_score(y_test, y_pred, average='weighted')
    rec = recall_score(y_test, y_pred, average='weighted')

    results.append({
        "Model": name,
        "Data": "Plaintext",
        "Accuracy": round(acc,4),
        "F1": round(f1,4),
        "Precision": round(prec,4),
        "Recall": round(rec,4),
        "TrainTime(s)": round(train_time,2),
        "TestTime(s)": round(test_time,2)
    })

    if name == "Logistic Regression":
        X_train_enc = encrypt_matrix(X_train_pca)
        X_test_enc = encrypt_matrix(X_test_pca)

        X_train_dec = decrypt_matrix(X_train_enc)
        X_test_dec = decrypt_matrix(X_test_enc)

        t0 = time.time()
        model.fit(X_train_dec, y_train)
        train_time = time.time() - t0

        t1 = time.time()
        y_pred = model.predict(X_test_dec)
        test_time = time.time() - t1

        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        prec = precision_score(y_test, y_pred, average='weighted')
        rec = recall_score(y_test, y_pred, average='weighted')

        results.append({
            "Model": name,
            "Data": "Encrypted",
            "Accuracy": round(acc,4),
            "F1": round(f1,4),
            "Precision": round(prec,4),
            "Recall": round(rec,4),
            "TrainTime(s)": round(train_time,2),
            "TestTime(s)": round(test_time,2)
        })

results_df = pd.DataFrame(results)
print(results_df)


***Training The Transfer Learning Models Using The Image Dataset***

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3, DenseNet121, MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [ ]:
image_size = (224, 224)
batch_size = 16
epochs = 100
data_dir = "/content/drive/MyDrive/Brain Tumor Dataset"
classes = sorted(os.listdir(data_dir))

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.15
)

train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)

num_classes = len(train_gen.class_indices)

In [ ]:
# --- TL models to compare ---
tl_models = {
    "VGG16": VGG16,
    "VGG19": VGG19,
    "ResNet50": ResNet50,
    "InceptionV3": InceptionV3,
    "DenseNet121": DenseNet121,
    "MobileNetV2": MobileNetV2
}

results = []

for name, base_model_class in tl_models.items():
    print(f"\nTraining {name}...")

    # Loading base model without top
    base_model = base_model_class(weights='imagenet', include_top=False, input_shape=(224,224,3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze base layers initially
    for layer in base_model.layers:
        layer.trainable = False

    # Compile
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Train
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs,
        verbose=1
    )

    val_gen.reset()
    y_true = val_gen.classes
    y_pred_probs = model.predict(val_gen, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    prec = precision_score(y_true, y_pred, average='weighted')
    rec = recall_score(y_true, y_pred, average='weighted')

    results.append({
        "Model": name,
        "Accuracy": round(acc,4),
        "F1": round(f1,4),
        "Precision": round(prec,4),
        "Recall": round(rec,4)
    })

# --- Save results ---
results_df = pd.DataFrame(results)
print("\nTransfer Learning Models Comparison:")
print(results_df)

***Implementing Hybrid Model (Image+text)***

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses, metrics
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
DATASET_DIR = "/content/drive/MyDrive/Brain Tumor Dataset/Testing"
CSV_FILE = "/content/dataset_features_golden.csv"
IMG_SIZE = (128, 128)
BATCH_SIZE = 16
EPOCHS = 100
RANDOM_SEED = 42
PCA_COMPONENTS = 112

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

df = pd.read_csv(CSV_FILE)
df = df.fillna(df.mean(numeric_only=True))
X_csv = df.values.astype(np.float32)

scaler = StandardScaler()
X_csv_scaled = scaler.fit_transform(X_csv)

if X_csv_scaled.shape[1] > PCA_COMPONENTS:
    pca = PCA(n_components=PCA_COMPONENTS, random_state=RANDOM_SEED)
    X_csv_pca = pca.fit_transform(X_csv_scaled)
else:
    X_csv_pca = X_csv_scaled

print("CSV features shape:", X_csv_pca.shape)

class_names = sorted(os.listdir(DATASET_DIR))
num_classes = len(class_names)

image_paths = []
labels = []

for label, cls in enumerate(class_names):
    cls_folder = os.path.join(DATASET_DIR, cls)
    for fname in sorted(os.listdir(cls_folder)):
        fpath = os.path.join(cls_folder, fname)
        image_paths.append(fpath)
        labels.append(label)

print("Total images found:", len(image_paths))
print("Total CSV rows:", X_csv_pca.shape[0])
assert len(image_paths) == X_csv_pca.shape[0], "Mismatch between images and CSV rows!"

def load_and_preprocess(path):
    img = load_img(path, target_size=IMG_SIZE)
    arr = img_to_array(img).astype("float32") / 255.0
    return arr

images = np.array([load_and_preprocess(p) for p in image_paths])
labels = np.array(labels)

print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

X_img_train, X_img_test, X_csv_train, X_csv_test, y_train, y_test = train_test_split(
    images, X_csv_pca, labels, test_size=0.2, random_state=RANDOM_SEED, stratify=labels
)

In [ ]:
def build_image_branch(input_shape=IMG_SIZE+(3,), embedding_dim=128):
    inp = layers.Input(shape=input_shape, name='img_input')
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inp)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(embedding_dim, activation='relu')(x)
    return models.Model(inp, x, name='image_branch')

def build_csv_branch(input_dim, embedding_dim=32):
    inp = layers.Input(shape=(input_dim,), name='csv_input')
    x = layers.Dense(128, activation='relu')(inp)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(embedding_dim, activation='relu')(x)
    return models.Model(inp, x, name='csv_branch')

def build_hybrid_model(img_shape, csv_dim, num_classes):
    img_branch = build_image_branch(input_shape=img_shape, embedding_dim=128)
    csv_branch = build_csv_branch(input_dim=csv_dim, embedding_dim=32)

    fused = layers.Concatenate()([img_branch.output, csv_branch.output])
    x = layers.Dense(128, activation='relu')(fused)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation='relu')(x)
    out = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=[img_branch.input, csv_branch.input], outputs=out, name='hybrid_model')
    return model

model = build_hybrid_model(
    img_shape=IMG_SIZE+(3,),
    csv_dim=X_csv_pca.shape[1],
    num_classes=num_classes
)

model.compile(
    optimizer=optimizers.Adam(1e-4),
    loss=losses.SparseCategoricalCrossentropy(),
    metrics=[metrics.SparseCategoricalAccuracy()]
)

model.summary()

In [ ]:
history = model.fit(
    [X_img_train, X_csv_train],
    y_train,
    validation_data=([X_img_test, X_csv_test], y_test),
    epochs=1,
    batch_size=BATCH_SIZE,
    verbose=2
)

test_loss, test_acc = model.evaluate([X_img_test, X_csv_test], y_test, verbose=0)
print("Hybrid Model - Test Loss:", test_loss, "Test Accuracy:", test_acc)

***Implementing Homomorphic Encryption***

In [ ]:
df = pd.read_csv(csv_path)
df = df.fillna(df.mean(numeric_only=True))
X_csv = df.drop("label", axis=1).values
y_csv = df["label"].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_csv)
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y_csv, test_size=0.2, random_state=42
)
print(f"Original features shape: {X_csv.shape}")
print(f"PCA features shape: {X_pca.shape}")
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")
logreg = LogisticRegression(max_iter=500, multi_class='ovr')
logreg.fit(X_train, y_train)
joblib.dump(logreg, "logreg_csv_for_he.joblib")
acc = logreg.score(X_test, y_test)
print(f"Plain text accuracy: {acc:.4f}")
X_csv_test = X_test
y_test = y_test

# ---------- Prepare weights & bias ----------
weights = logreg.coef_.astype(np.float64)
bias = logreg.intercept_.astype(np.float64)

n_classes, D = weights.shape
print(f"LogReg weights shape: {weights.shape}, bias shape: {bias.shape}")

# ---------- TenSEAL CKKS context (client) ----------
# Use safe parameters: poly_modulus_degree must be power of two, >= 4096. 8192 is common.
def make_tenseal_context(poly_modulus_degree=8192, coeff_mod_bit_sizes=[60, 40, 40, 60], global_scale=2**40):
    ctx = ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=poly_modulus_degree,
        coeff_mod_bit_sizes=coeff_mod_bit_sizes
    )
    ctx.generate_galois_keys()
    ctx.global_scale = global_scale
    # IMPORTANT: in a real deployment, the client keeps the secret key and only shares public keys
    return ctx

ctx = make_tenseal_context()

# ---------- Choose a small batch to demo ----------
batch_size = min(20, X_csv_test.shape[0])
batch_idx = np.arange(batch_size)
X_batch = X_csv_test[batch_idx].astype(np.float64)
y_batch = y_test[batch_idx] if 'y_test' in globals() else None

# ---------- Encryption (client) ----------
enc_start = time.time()
enc_vectors = [ts.ckks_vector(ctx, X_batch[i].tolist()) for i in range(X_batch.shape[0])]
enc_time = time.time() - enc_start
print(f"Encrypted {X_batch.shape[0]} vectors in {enc_time:.3f} s")

# ---------- Server-side computation (plaintext weights) ----------
# For multiclass: compute encrypted logits for each class (one ciphertext per sample per class)
server_start = time.time()
enc_logits_per_sample = []  # list length B; each element will be list of ciphertext logits length C

for enc_vec in enc_vectors:
    enc_logits = []
    # compute encrypted dot for each class weight
    for c in range(n_classes):
        w_c = weights[c].tolist()   # plaintext list
        b_c = float(bias[c])
        # elementwise multiplication (ciphertext * plaintext list)
        enc_prod = enc_vec * w_c     # returns a CKKSVector ciphertext
        # sum all slots to get scalar ciphertext (approx dot product)
        enc_dot = enc_prod.sum()     # CKKSVector.sum() reduces to a single-slot ciphertext
        enc_logit = enc_dot + b_c    # add plaintext bias
        enc_logits.append(enc_logit)
    enc_logits_per_sample.append(enc_logits)

server_time = time.time() - server_start
print(f"Server computed encrypted logits in {server_time:.3f} s")

# ---------- Client-side decryption ----------
dec_start = time.time()
decoded_logits = np.zeros((X_batch.shape[0], n_classes), dtype=np.float64)
for i in range(X_batch.shape[0]):
    for c in range(n_classes):
        val = enc_logits_per_sample[i][c].decrypt()[0]
        decoded_logits[i, c] = float(val)
dec_time = time.time() - dec_start
print(f"Decrypted logits for batch in {dec_time:.3f} s")

# ---------- Compute predictions & compare to plaintext ----------
def softmax_rows(logit_matrix):
    ex = np.exp(logit_matrix - np.max(logit_matrix, axis=1, keepdims=True))
    return ex / (np.sum(ex, axis=1, keepdims=True) + 1e-12)

probs_he = softmax_rows(decoded_logits)
pred_he = np.argmax(probs_he, axis=1)

# Plaintext predictions for batch
pred_plain = logreg.predict(X_batch)

print("HE batch predictions:", pred_he)
print("Plain batch predictions:", pred_plain)

agree = np.mean(pred_he == pred_plain)
print(f"Agreement between HE predictions and plaintext logistic predictions: {agree*100:.2f}%")

# If true labels available, compute HE accuracy
if y_batch is not None:
    he_acc = accuracy_score(y_batch, pred_he)
    plain_acc = accuracy_score(y_batch, pred_plain)
    print(f"HE accuracy on batch: {he_acc:.4f}, Plain accuracy on batch: {plain_acc:.4f}")

# ---------- Timings summary ----------
print(f"Timings (s) -> encrypt: {enc_time:.3f}, server: {server_time:.3f}, decrypt: {dec_time:.3f} for batch size {X_batch.shape[0]}")


***Extracting LBP Mean and GLCM Distribution Across Dataset***

In [ ]:
import pandas as pd

df = pd.read_csv("/content/braintumor_features.csv")
label_map = {0: "No Tumor", 1: "Glioma", 2: "Meningioma", 3: "Pituitary"}
df['label_name'] = df['label'].map(label_map)

selected_features = [
    'T1_ABD_lbp_mean',
    'T1_ABD_lbp_hist_0',
    'T1_ABD_glcm_contrast',
    'T1_ABD_glcm_energy',
    'T1_ABD_edge_density',
    'T1_ABD_gradient_mean'
]

summary_list = []
for feature in selected_features:
    row = {'Feature': feature}
    for label_val, label_name in label_map.items():
        mean = df[df['label'] == label_val][feature].mean()
        std = df[df['label'] == label_val][feature].std()
        row[label_name] = f"{mean:.3f} ± {std:.3f}"
    summary_list.append(row)

summary_table = pd.DataFrame(summary_list)

# LBP mean distribution across classes
sns.boxplot(x='label_name', y='T1_ABD_lbp_mean', data=df)
plt.title("LBP Mean Distribution Across Tumor Classes")
plt.xlabel("Tumor Class")
plt.ylabel("LBP Mean")
plt.show()

# GLCM contrast distribution
sns.boxplot(x='label_name', y='T1_ABD_glcm_contrast', data=df)
plt.title("GLCM Contrast Distribution Across Tumor Classes")
plt.xlabel("Tumor Class")
plt.ylabel("GLCM Contrast")
plt.show()